<a href="https://colab.research.google.com/github/luoshaoyang/cs224w_final_proj/blob/main/Colab_FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Visible Masking/Watermark Removal with Graph Neural Network

# Setup
* Import Libraries
* Handy Functions<br>
 1. loadimage
 2. image2NN
 3. NN2Image
 4. plotImage
 5. image2GrayScale
 6. image2Binary
 7. overlayerImg
 8. overlayImg1OverImg2


In [ ]:
# Packages

In [ ]:
# Functions

#Data Processing
* Read in the datasets
* Create masked/watermarked datasets
* Split for training and testing

In [ ]:
# Load DataSet

In [ ]:
# Create masked datasets

In [ ]:
# Split data for training and testing

# Model Build
* Learn GNN embedding
* Apply GCN to predict<br>
  1. GCN training
  2. Accuracy measurement
* Apply label propagation to predict<br>
  1. Edge weight training
  2. propagation iteration
  3. Accuracy measurement

In [ ]:
# GNN embedding learning

In [ ]:
# GCN Model

In [ ]:
# Label Propagation